# Эксперимент 1

Метрика для сравнения - средняя ширина доверительного интервала при проведенных 1000 AA тестов. Тесты, которые будем сравнивать:

1. T-test
2. Bootstrap (центральный)
3. Normed Bootstrap (центральный)
4. Cuped

Также выведем для каждого теста реально достигнутый уровень значимости. Давайте сначала реализуем каждый из тестов. 

Всюду $\alpha = 0.05$.

In [1]:
from collections import namedtuple
import scipy.stats as sps
import statsmodels.stats.api as sms
import numpy as np

ExperimentComparisonResults = namedtuple('ExperimentComparisonResults',
                                        ['pvalue', 'effect', 'ci_length'])

ExperimentComparisonResultsBootstrap = namedtuple('ExperimentComparisonResultsBootstrap',
                                                ['effect_flag', 'effect', 'ci_length'])

In [2]:
def ttest(test, control, alpha=0.05):
    test_result = sms.CompareMeans(sms.DescrStatsW(test), sms.DescrStatsW(control))
    left_bound, right_bound = test_result.tconfint_diff(alpha=alpha, alternative='two-sided', usevar='unequal')

    ci_length = right_bound - left_bound
    pvalue = sps.ttest_ind(test, control, equal_var=False).pvalue
    effect = np.mean(test) - np.mean(control)
    return ExperimentComparisonResults(pvalue, effect, ci_length)


def cuped(control_before, test_before, control, test, alpha=0.05):
    # Считаем teta с помощью теста и контроля на предпериоде (у них должны быть примерно одинаковые средние, размеры выборок не обязательно равны)
    teta = ((np.cov(test, test_before)[0, 1] + np.cov(control, control_before)[0, 1]) \
        / (np.var(test_before) + np.var(control_before)))
    
    # Считаем новые тест и контроль выборки
    new_test = test - teta * test_before
    new_control = control - teta * control_before
    
    return ttest(new_test, new_control, alpha)

In [3]:
def central_bootstrap(control, test, *args, n=3000, alpha=0.05, normed=False):
    # Шаг 0. Вычислим длины тестовой и контрольной выборки
    M_1 = len(control)
    M_2 = len(test)
    
    # Шаг 1.1. Если нормированный бутстрап, то учтем это.
    if normed:
        control_before, test_before = args

        # Шаг 1.2. Считаем разницу выборочных квантилей у теста и контроля.
        effect = test.mean() - (test_before.mean() / control_before.mean()) * control.mean()

        # Шаг 1.3. Генерируем индексы
        control_indices = np.arange(M_1)
        test_indices = np.arange(M_2)
        
        control_indices_sample = np.random.choice(control_indices, size=(n, M_1), replace=True)
        test_indices_sample = np.random.choice(test_indices, size=(n, M_2), replace=True)
        
        # Шаг 2.1. Генерируем n псевдовыборок из контрольной (и тестовой) группы.
        bootstrap_values_c = control[control_indices_sample]
        bootstrap_values_t = test[test_indices_sample]

        # Шаг 2.2. Генерируем n псевдовыборок из контрольной (и тестовой) группы на предпериоде.
        bootstrap_values_c_before = control_before[control_indices_sample]
        bootstrap_values_t_before = test_before[test_indices_sample]

        # Шаг 2.3. Считаем n выборочных средних у n псевдовыборок из контрольной (тестовой) группы.
        bootstrap_metrics_c = np.mean(bootstrap_values_c, axis=1)
        bootstrap_metrics_t = np.mean(bootstrap_values_t, axis=1)

        # Шаг 2.4. Считаем n выборочных средних у n псевдовыборок из контрольной (тестовой) группы на предпериоде.
        bootstrap_metrics_c_before = np.mean(bootstrap_values_c_before, axis=1)
        bootstrap_metrics_t_before = np.mean(bootstrap_values_t_before, axis=1)

        # Шаг 3. Считаем разницу полученных метрик
        bootstrap_stats = bootstrap_metrics_t - (bootstrap_metrics_t_before / bootstrap_metrics_c_before) * bootstrap_metrics_c

        # Шаг 4. Строим доверительный интервал методом Центрального бутстрапа.
        left = 2 * effect - np.quantile(a=bootstrap_stats, q=1 - alpha / 2)
        right = 2 * effect - np.quantile(a=bootstrap_stats, q=alpha / 2)

        # Шаг 5. Смотрим ширину доверительного интервала и определяем наличие эффекта.
        ci_length = right - left

        # Шаг 6. Определяем наличие эффекта
        effect_flag = not (left < 0 < right)

        return ExperimentComparisonResultsBootstrap(effect_flag, effect, ci_length)
    
    else:
        # Шаг 1.2. Считаем разницу выборочных средних у теста и контроля.
        effect = test.mean() - control.mean()

        # Шаг 2.1. Генерируем n псевдовыборок из контрольной (и тестовой) группы.
        bootstrap_values_c = np.random.choice(control, (n, M_1), replace=True)
        bootstrap_values_t = np.random.choice(test, (n, M_2), replace=True)

        # Шаг 2.2. Считаем n выборочных средних у n псевдовыборок из контрольной (тестовой) группы .
        bootstrap_metrics_c = np.mean(bootstrap_values_c, axis=1)
        bootstrap_metrics_t = np.mean(bootstrap_values_t, axis=1)

        # Шаг 3. Считаем разницу полученных метрик
        bootstrap_stats = bootstrap_metrics_t - bootstrap_metrics_c

        # Шаг 4. Строим доверительный интервал методом Центрального бутстрапа.
        left = 2 * effect - np.quantile(a=bootstrap_stats, q=1 - alpha / 2)
        right = 2 * effect - np.quantile(a=bootstrap_stats, q=alpha / 2)

        # Шаг 5. Смотрим ширину доверительного интервала и определяем наличие эффекта.
        ci_length = right - left

        # Шаг 6. Определяем наличие эффекта
        effect_flag = not (left < 0 < right)

        return ExperimentComparisonResultsBootstrap(effect_flag, effect, ci_length)

In [4]:
ttest_counter_effect = 0
cuped_counter_effect = 0
central_bootstrap_counter_effect = 0
central_normed_bootstrap_counter_effect = 0

ttest_ci_length = []
cuped_ci_length = []
central_bootstrap_ci_length = []
central_normed_bootstrap_ci_length = []

N = 1500

for i in range(N):
    control_before = sps.expon.rvs(scale=100, size=1000)
    test_before = sps.expon.rvs(scale=100, size=1000)

    control = control_before + sps.expon.rvs(scale=5, size=1000)
    test = test_before + sps.expon.rvs(scale=5, size=1000)

    ttest_result = ttest(test, control)
    cuped_result = cuped(control_before, test_before, control, test)
    central_bootstrap_result = central_bootstrap(control, test)
    central_normed_bootstrap_result = central_bootstrap(control, test, control_before, test_before, normed=True)

    ttest_counter_effect += ttest_result.pvalue < 0.05
    cuped_counter_effect += cuped_result.pvalue < 0.05
    central_bootstrap_counter_effect += central_bootstrap_result.effect_flag
    central_normed_bootstrap_counter_effect += central_normed_bootstrap_result.effect_flag

    ttest_ci_length.append(ttest_result.ci_length)
    cuped_ci_length.append(cuped_result.ci_length)
    central_bootstrap_ci_length.append(central_bootstrap_result.ci_length)
    central_normed_bootstrap_ci_length.append(central_normed_bootstrap_result.ci_length)

print(f"Реальный достигнутый уровень значимости для T-test: {round(ttest_counter_effect / N, 4)}",
    f"Реальный достигнутый уровень значимости для Cuped: {round(cuped_counter_effect / N, 4)}",
    f"Реальный достигнутый уровень значимости для бутстрапа: {round(central_bootstrap_counter_effect / N, 4)}",
    f"Реальный достигнутый уровень значимости для нормированного бутстрапа: {round(central_normed_bootstrap_counter_effect / N, 4)}",
      sep='\n')

print('-------------------------------------------------------------------------------')

print(f"Среднее длин доверительных интервалов для T-test: {np.array(ttest_ci_length).mean().round(4)}",
    f"Среднее длин доверительных интервалов для Cuped: {np.array(cuped_ci_length).mean().round(4)}",
    f"Среднее длин доверительных интервалов для бутстрапа: {np.array(central_bootstrap_ci_length).mean().round(4)}",
    f"Среднее длин доверительных интервалов для нормированного бутстрапа: {np.array(central_normed_bootstrap_ci_length).mean().round(4)}",
     sep='\n')

Реальный достигнутый уровень значимости для T-test: 0.0573
Реальный достигнутый уровень значимости для Cuped: 0.048
Реальный достигнутый уровень значимости для бутстрапа: 0.0573
Реальный достигнутый уровень значимости для нормированного бутстрапа: 0.0547
-------------------------------------------------------------------------------
Среднее длин доверительных интервалов для T-test: 17.5555
Среднее длин доверительных интервалов для Cuped: 0.877
Среднее длин доверительных интервалов для бутстрапа: 17.5014
Среднее длин доверительных интервалов для нормированного бутстрапа: 1.2396
